In [1]:
import sqlite3
import pandas as pd
import os
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
# import musicbrainzngs

os.chdir('../')
os.chdir('../')

from plagdet.src.defaults import *

In [2]:
import magenta.music as mm
from note_seq.midi_io import note_sequence_to_midi_file
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
example = 'plagdet\\data\\midi_databases\\aa_midis\\No-Scrubs.mid'

MIDIPlayer(example, 500, styler=cifka_advanced, title='My Player')



### class which does the remaining code

In [18]:
from plagdet.src.embeddings.music_vae.enc import Mel2barEncoder

example = 'plagdet\\data\\midi_databases\\aa_midis\\No-Scrubs.mid'

encoder = Mel2barEncoder()
vecs = encoder.encode_midis([example])
print(vecs)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


{'plagdet\\data\\midi_databases\\aa_midis\\No-Scrubs.mid': [array([-1.35094035e+00,  6.46137178e-01, -5.78408957e-01,  1.95681453e-01,
       -5.17798439e-02,  2.09903646e+00,  7.59442151e-02, -1.16622329e+00,
       -1.16075385e+00, -9.88013744e-02, -2.53025759e-02,  1.34097743e+00,
       -2.87191331e-01,  1.01577878e-01, -1.09535050e+00, -1.58200836e+00,
       -1.90441430e-01, -6.62992120e-01,  2.09394979e+00, -2.29629204e-01,
        3.69592518e-01,  1.57508469e+00,  1.01595783e+00,  1.01107419e+00,
       -8.29479516e-01,  1.33093655e+00,  5.46198547e-01, -1.52076638e+00,
        3.41289163e-01,  1.52619612e+00, -7.18034744e-01, -9.19515416e-02,
       -8.29876065e-02,  7.22652733e-01,  3.66892219e-01, -1.09445500e+00,
       -6.06732607e-01,  1.15001249e+00,  2.86111891e-01,  5.53921759e-01,
        2.53617227e-01,  2.45015562e-01, -1.55268073e+00,  8.41493309e-01,
        5.59762478e-01,  1.91322416e-01,  5.72187364e-01,  3.30462575e-01,
       -9.90516171e-02,  6.08561754e-01,

In [19]:
type(vecs)

dict

### load model

In [5]:
## attempting to change the config

import collections

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import data
from magenta.models.music_vae import data_hierarchical
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae.base_model import MusicVAE
import note_seq

HParams = contrib_training.HParams

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

  def values(self):
    return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
  config_dict = config.values()
  config_dict.update(update_dict)
  return Config(**config_dict)

In [6]:
mel_2bar_config = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.CategoricalLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=32,  # 2 bars w/ 16 steps per bar
            z_size=512,
            enc_rnn_size=[2048],
            dec_rnn_size=[2048, 2048, 2048],
            free_bits=0,
            max_beta=0.5,
            beta_rate=0.99999,
            sampling_schedule='inverse_sigmoid',
            sampling_rate=1000,
        )),
    note_sequence_augmenter=data.NoteSequenceAugmenter(transpose_range=(-5, 5)),
    data_converter=data.OneHotMelodyConverter(
        valid_programs=data.MEL_PROGRAMS,
        skip_polyphony=False,
        max_bars=100,  # Truncate long melodies before slicing.
        slice_bars=2,
        steps_per_quarter=4),
    train_examples_path=None,
    eval_examples_path=None,
)

In [7]:
# mel_2bar_config = configs.CONFIG_MAP[MUSICVAE_2BAR_BIG_CONFIG]
mel_2bar = TrainedModel(mel_2bar_config, batch_size=4, checkpoint_dir_or_path=MUSICVAE_2BAR_BIG_PATH)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


### extract melodies

In [8]:
input_mel_midi_data = [example]
# mel_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_mel_midi_data]
print(input_mel_midi_data)
mel_input_seqs = []
for midi_path in input_mel_midi_data:
    with open(midi_path, 'rb') as midi_file:
        midi_data = midi_file.read()
        mel_input_seqs.append(mm.midi_to_sequence_proto(midi_data))
  
extracted_mels = []
for ns in mel_input_seqs:
  extracted_mels.extend(
      mel_2bar_config.data_converter.from_tensors(
          mel_2bar_config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_mels):
  print("Melody", i)

['plagdet\\data\\midi_databases\\aa_midis\\No-Scrubs.mid']
Melody 0
Melody 1
Melody 2
Melody 3
Melody 4


In [9]:
note_sequence_to_midi_file(extracted_mels[2], 'midi.mid')
MIDIPlayer('midi.mid', 500, styler=cifka_advanced, title='My Player')

In [10]:
for melody in extracted_mels:
    note_sequence_to_midi_file(melody, 'midi.mid')
    MIDIPlayer('midi.mid', 500, styler=cifka_advanced, title='My Player')


### encode notesequences

In [11]:
note_sequences = extracted_mels.copy()
inputs = []
controls = []
lengths = []
note_sequence = note_sequences[4]
extracted_tensors = mel_2bar._config.data_converter.to_tensors(note_sequence)
if not extracted_tensors.inputs:
    raise Exception('No examples extracted from NoteSequence: %s' % note_sequence)
# if len(extracted_tensors.inputs) > 1:
#     raise Exception('Multiple (%d) examples extracted from NoteSequence: %s' % (len(extracted_tensors.inputs), note_sequence))
print(len(extracted_tensors.inputs))
i = 0
inputs.append(extracted_tensors.inputs[i])
controls.append(extracted_tensors.controls[i])
lengths.append(extracted_tensors.lengths[i])

vec = mel_2bar.encode_tensors(inputs, lengths, controls)[0]

1


In [12]:
controls

[array([], shape=(32, 0), dtype=float64),
 array([], shape=(0, 0), dtype=float64),
 array([], shape=(0, 0), dtype=float64),
 array([], shape=(0, 0), dtype=float64)]

In [13]:
vec

array([[ 1.2703465 , -0.17577027,  0.82652235,  0.7687316 , -0.48307663,
        -0.36791998,  1.0223026 ,  0.11748298, -0.8425244 ,  0.03611445,
        -0.20947367,  0.7165921 ,  0.41503483, -0.9709201 , -0.30103672,
         0.04286458,  0.62336254,  0.6311137 , -0.6316656 , -1.2342647 ,
         1.381911  ,  0.36926547,  0.41194835,  0.20727238, -0.75721854,
        -0.46802577, -0.520006  , -0.2189994 , -0.6626461 ,  1.2195333 ,
        -0.08931445,  0.47512624, -0.28996602,  0.7994713 , -1.1594001 ,
        -0.05201854,  0.21887097, -0.5315405 ,  1.3423245 , -0.16410466,
         0.48330042,  1.4857606 ,  0.5454382 ,  2.04979   , -1.722981  ,
         1.4384617 ,  0.50009096, -1.5301406 , -0.7674563 , -0.8435924 ,
         1.2720417 ,  0.52766216,  1.9213494 , -1.14406   ,  1.2073314 ,
        -1.7447897 , -0.13274527,  1.8456262 ,  1.0555029 ,  0.5227972 ,
        -0.7037474 ,  0.08594331,  0.4597632 , -1.3230457 ,  0.10678634,
         0.41631755,  0.43091345,  0.4568668 , -1.5

In [14]:
vec

array([[ 1.2703465 , -0.17577027,  0.82652235,  0.7687316 , -0.48307663,
        -0.36791998,  1.0223026 ,  0.11748298, -0.8425244 ,  0.03611445,
        -0.20947367,  0.7165921 ,  0.41503483, -0.9709201 , -0.30103672,
         0.04286458,  0.62336254,  0.6311137 , -0.6316656 , -1.2342647 ,
         1.381911  ,  0.36926547,  0.41194835,  0.20727238, -0.75721854,
        -0.46802577, -0.520006  , -0.2189994 , -0.6626461 ,  1.2195333 ,
        -0.08931445,  0.47512624, -0.28996602,  0.7994713 , -1.1594001 ,
        -0.05201854,  0.21887097, -0.5315405 ,  1.3423245 , -0.16410466,
         0.48330042,  1.4857606 ,  0.5454382 ,  2.04979   , -1.722981  ,
         1.4384617 ,  0.50009096, -1.5301406 , -0.7674563 , -0.8435924 ,
         1.2720417 ,  0.52766216,  1.9213494 , -1.14406   ,  1.2073314 ,
        -1.7447897 , -0.13274527,  1.8456262 ,  1.0555029 ,  0.5227972 ,
        -0.7037474 ,  0.08594331,  0.4597632 , -1.3230457 ,  0.10678634,
         0.41631755,  0.43091345,  0.4568668 , -1.5

In [15]:
note_sequences = mel_input_seqs.copy()

inputs = []
controls = []
lengths = []

vecs = []

for note_sequence in note_sequences:
    extracted_tensors = mel_2bar._config.data_converter.to_tensors(note_sequence)
    if not extracted_tensors.inputs:
        raise Exception('No examples extracted from NoteSequence: %s' % note_sequence)
    inputs.append(extracted_tensors.inputs[0])
    controls.append(extracted_tensors.controls[0])
    lengths.append(extracted_tensors.lengths[0])
    vecs.append(mel_2bar.encode_tensors(inputs, lengths, controls))

In [16]:
encoded_mels = mel_2bar.encode(mel_input_seqs)
print(encoded_mels)

MultipleExtractedExamplesError: Multiple (5) examples extracted from NoteSequence: ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
tempos {
  qpm: 93.000186000372
}
notes {
  pitch: 56
  velocity: 75
  start_time: 2.74193
  end_time: 3.77821825
  program: 25
}
notes {
  pitch: 64
  velocity: 89
  start_time: 3.0645100000000003
  end_time: 3.77821825
  program: 25
}
notes {
  pitch: 49
  velocity: 91
  start_time: 2.5806400000000003
  end_time: 3.8185407500000004
  program: 25
}
notes {
  pitch: 59
  velocity: 73
  start_time: 2.90322
  end_time: 3.85886325
  program: 25
}
notes {
  pitch: 44
  velocity: 91
  start_time: 3.87096
  end_time: 4.03628225
  program: 25
}
notes {
  pitch: 56
  velocity: 75
  start_time: 4.03225
  end_time: 5.0685382500000005
  program: 25
}
notes {
  pitch: 63
  velocity: 89
  start_time: 4.354830000000001
  end_time: 5.0685382500000005
  program: 25
}
notes {
  pitch: 59
  velocity: 73
  start_time: 4.1935400000000005
  end_time: 

### trying my own class

In [ ]:
os.getcwd()

'c:\\Users\\cunn2\\OneDrive\\DSML\\Project\\thesis-repo'

In [ ]:
from plagdet.src.embeddings.music_vae.encoder import MusicVAEncoder

encoder = MusicVAEncoder(example)
encoded_mels = encoder.encode_melodies()
print(encoded_mels)

Loaded MIDI file: plagdet\data\midi_databases\aa_midis\No-Scrubs.mid
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\contrib\rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.
INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


NoExtractedExamplesError: No examples extracted from NoteSequence: ticks_per_quarter: 220
tempos {
  qpm: 120.0
}
notes {
  pitch: 49
  velocity: 100
  start_time: 2.0
  end_time: 2.125
}
notes {
  pitch: 56
  velocity: 100
  start_time: 2.125
  end_time: 2.25
}
notes {
  pitch: 59
  velocity: 100
  start_time: 2.25
  end_time: 2.375
}
notes {
  pitch: 64
  velocity: 100
  start_time: 2.375
  end_time: 2.9375
}
notes {
  pitch: 44
  velocity: 100
  start_time: 3.0
  end_time: 3.125
}
notes {
  pitch: 56
  velocity: 100
  start_time: 3.125
  end_time: 3.25
}
notes {
  pitch: 59
  velocity: 100
  start_time: 3.25
  end_time: 3.375
}
notes {
  pitch: 63
  velocity: 100
  start_time: 3.375
  end_time: 3.9375
}
total_time: 3.9375
